# Librerias

In [2]:
#Importemos las librerías Necesarias:
import scipy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, time

In [3]:
from sklearn.pipeline import *
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.neural_network import *
from sklearn.impute import *

In [4]:
# Carguemos los archivos de drive (en caso de usar colab)
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


***

# Main Code

Tras analizar el CSV, se ha visto que hay ceritos datos que están mal imputados por lo que realizaremos una corrección sobre ellos antes de comenzar con el problema.<br>
Esto se realiza porque estos datos no son outliers si no datos en los ha habido un error (humano lo mas probable) al introducirlos. 

In [5]:
# Empecemos cargando la información:
try:
  df = pd.read_csv('gdrive/My Drive/winequality-white-mod.csv', sep=';')
except:
  df = pd.read_csv('winequality-white-mod.csv', sep=';')
cols = list(df.columns)

# Adecuemos los missing values:
df[df=='?'] = np.nan

# Adecuemos el tipo de dato:
for row in df:
    df[row] = pd.to_numeric(df[row])
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,45.00,45.0,170.0,1001.0000,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,49.00,14.0,132.0,994.0000,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.05,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,58.00,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,58.00,47.0,186.0,0.9956,3.19,0.40,9.9,6


La falla en la imputación se ha localizado en la columna Density, como se puede ver en los patrones 0 y 1, están cerca o por encima del millar los cual, para el vino, no tiene sentido.<br>
Esto se solucionará dividiendo estas entradas de la variable desity por 1000.

In [6]:
# Para ello iteraremos sobre todas las instancias de la variable desity:
for i in df.index:
    if df['density'][i] > 100:
        df['density'][i] /= 1000

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Una vez corregido este fallo en la imputación de los datos, pasemos a resolver el ejercicio:

In [7]:
# Creemos ciertas instancias necesarias:
X = df.drop(cols[-1], axis=1)
y = df[cols[-1]]
seed = random.seed(time.time())

# Seleccionamos los modelos:
imptr = KNNImputer() # de imputación
sclr = RobustScaler() # de escalado
model = MLPClassifier(random_state=seed) # de clasificación
cval = StratifiedKFold(n_splits=20, random_state=seed, shuffle=True) # de ajuste

In [44]:
# Creamos el flujo de trabajo:
pipe = Pipeline([
    ('imputer', imptr),
    ('scaler', sclr),
    ('MLPClassifier', model)
])

# Creemos la estructura de ajuste de Hiperparametros:
space = [{
    'MLPClassifier__solver': ['lbfgs', 'adam'], # ‘lbfgs’, ‘sgd’, ‘adam’ 
    'MLPClassifier__max_iter': [1000, 2000, 3000, 5000],
    'MLPClassifier__activation' : ['relu', 'logistic'], # ‘identity’, ‘logistic’, ‘tanh’, ‘relu’
    'MLPClassifier__hidden_layer_sizes':[(10,),(20,)],
}]

h_adj = GridSearchCV(
    estimator=pipe, 
    param_grid=space,
    cv=cval,
    n_jobs=-1
)

t_0 = time.time()
h_adj.fit(X, y)
t_1 = time.time()

res = h_adj.best_params_


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=20.
  UserWarning,


In [45]:
dt = t_1 - t_0
H, dt = dt // 3600, dt % 3600
m, dt = dt // 60, dt % 60
print(f'Tiempo de ejecucion: {H} horas, {m} minutos y {dt} segundos\n')
for item in list(res.keys()):
  print(f">{str(item).split('__')[-1].replace('_',' ')}:\n>>{res[item]}\n")

Tiempo de ejecucion: 1.0 horas, 0.0 minutos y 3.637521743774414 segundos

>activation:
>>logistic

>hidden layer sizes:
>>(10,)

>max iter:
>>5000

>solver:
>>adam



Como se puede apreciar en el resultado de la ultima celda, el tiempo de ejecucion para *ecualizar* este modelo de rpediccion ha resultado ser curiosamente largo por lo que se deduce que pretender estimar el modelo optimo añadiendo una tercera función de activación puede ser terriblemente largo, sendo asi (pues ya se ha intentado) dejaremos aqui el ejercicio.<br>

A modo de conclusion, pasaremos a coentar los resultados:
   * La funcion oiptima de activacion ha resultado ser la Logaritmica.
   * El numero optimo de capas ocultas.
   * La cantidad optima de iteraciones, 5000.
   * Y el resolutor para los poesos de las neuronas el adam.

Como dato comentar que al establecer como única funcion de activación la función relu, la cantidad optima de iter


***

## Notas:

> Nota 1

## Bibliografia:

   * Stack Overflow: https://stackoverflow.com
   * Scikit Learn: https://scikit-learn.org

### Autor

   - **Name:** Elidas
   - **Email:** pyro.elidas@gmail.com
   - **Python version:** 3.9.1
   - **Date:** ${DATE}